In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import dill, pickle
import tensorflow as tf
import pandas as pd
import glob
import os

from tqdm.notebook import tqdm 
from sklearn.preprocessing import robust_scale
from sklearn.model_selection import ParameterGrid


from rainbow.agent import Rainbow
from rainbow.utils.memories import RNNReplayMemory
from itertools import product

import sys
import gym_trading_env
import nest_asyncio

tf.random.set_seed(
    42
)

def reload_agent(
    modelname = 'prio20hNewReward',
    batch_size = 1024,
    train_every = 10
    ):
    with open(modelname + "/model.pkl", "rb") as file:
        agent = dill.load(file)

    with open(modelname + "/replay_memory.pkl", "rb") as file:
        agent.replay_memory = dill.load(file)
        
    agent.model = tf.keras.models.load_model(modelname)
    agent.target_model = tf.keras.models.load_model(modelname)

    #agent.replay_capacity = replay_capacity
    agent.batch_size = batch_size
    agent.train_every = train_every

    return agent

def add_features(df):
    # df = df[:2000].copy()
    
    df["feature_close"] = robust_scale(df["close"].pct_change(fill_method=None))
    df["feature_open"] = robust_scale(df["open"]/df["close"])
    df["feature_high"] = robust_scale(df["high"]/df["close"])
    df["feature_low"] = robust_scale(df["low"]/df["close"])
    df["feature_volume"] = robust_scale(df["volume"] / df["volume"].rolling(7*24).max())
    df.dropna(inplace= True) # Clean your data !
    return df


def reward_function(history):
    position = history["position", -1]
    data_close = history["data_close", -1]
    last_data_close = history["data_close", -2]    

    if position == 0:
        reward =  last_data_close - data_close
    if position == 1:
        reward =  data_close - last_data_close

    # h = history[-1]

    # print(h["step"], h["position"], h["data_close"], h["portfolio_valuation"], reward)

    return reward*3

    

    # return 800*np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2]) #log (p_t / p_t-1 )

def max_drawdown(history):
    networth_array = history['portfolio_valuation']
    _max_networth = networth_array[0]
    _max_drawdown = 0
    for networth in networth_array:
        if networth > _max_networth:
            _max_networth = networth
        drawdown = ( networth - _max_networth ) / _max_networth
        if drawdown < _max_drawdown:
            _max_drawdown = drawdown
    return f"{_max_drawdown*100:5.2f}%"

def make_env(df):
    env = gym.make(
        "TradingEnv",
        df=df,
        windows= 15,
        positions = [0, 1], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 0,
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        reward_function = reward_function,
        portfolio_initial_value = 1000, # here, in USDT
        
        verbose= 1,
    )
    env.unwrapped.add_metric('Position Changes', lambda history : f"{ 100*np.sum(np.diff(history['position']) != 0)/len(history['position']):5.2f}%" )
    env.unwrapped.add_metric('Max Drawdown', max_drawdown)
    return env

def get_env(dataset = '2017'):

    df = pd.read_pickle("data/processed/validation/binance-BTCUSDT-5m-5-from-dez-2017-until-dez-2018.pkl")
    if dataset == '2024':
        df = pd.read_pickle("data/processed/validation/binance-BTCUSDT-5m-0-2024.pkl")
    elif dataset == 'all':
        df = pd.read_pickle("data/raw/binance-BTCUSDT-5m-since-2017.pkl")

    df = add_features(df)
    env = make_env(df)

    return env


# training_envs = gym.vector.SyncVectorEnv([lambda: make_env("data/processed/training/*.pkl") for _ in range(6)])
# validation_envs = gym.vector.SyncVectorEnv([lambda: make_env("data/processed/validation/*.pkl") for _ in range(6)])

def evaluation(train=True):
    print("___________________________________________ VALIDATION ___________________________________________")
    done, truncated = False, False
    val_obs, info = env.reset()
    while not done and not truncated:
        action = agent.pick_action(val_obs)
        next_obs, reward, done, truncated, info = env.step(action)
        agent.store_replay(next_obs, action, reward, next_obs, done, truncated)
        if train:
            agent.train()
        if done or truncated:
            print('saving')
            env.unwrapped.save_for_render(dir = "render_logs")
        val_obs = next_obs

In [4]:
param_grid = ParameterGrid({
        'batch_size': [64, 128, 256], 
        'train_every': [64, 128, 256]
    })

In [5]:
for p in param_grid:
    print(p)

{'batch_size': 64, 'train_every': 64}
{'batch_size': 64, 'train_every': 128}
{'batch_size': 64, 'train_every': 256}
{'batch_size': 128, 'train_every': 64}
{'batch_size': 128, 'train_every': 128}
{'batch_size': 128, 'train_every': 256}
{'batch_size': 256, 'train_every': 64}
{'batch_size': 256, 'train_every': 128}
{'batch_size': 256, 'train_every': 256}


In [3]:
for params in param_grid:
    print('----------')
    print(params)
    print('----------')
    agent = reload_agent(
        batch_size=params['batch_size'],
        train_every=params['train_every']
    )
    agent.epsilon_function = lambda episode, step : 0
    
    env = get_env("2017")

    evaluation()

    list_of_files = glob.glob('render_logs/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    os.rename(latest_file, 'render_logs/' + f'batch_size_{params["batch_size"]}_train_every_{params["train_every"]}.pkl')

    del agent

----------
{'batch_size': 64, 'train_every': 64}
----------


2024-06-04 22:37:16.685756: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-04 22:37:16.919795: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-04 22:37:16.919882: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-04 22:37:16.923255: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-04 22:37:16.923331: I external/local_xla/xla/stream_executor

___________________________________________ VALIDATION ___________________________________________


2024-06-04 22:37:55.428477: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-06-04 22:38:02.957677: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fdfa9f9fad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-04 22:38:02.957715: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 980 Ti, Compute Capability 5.2
2024-06-04 22:38:02.973577: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1717533483.058119  215814 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Market Return : -64.43%   |   Portfolio Return : 165.31%   |   Position Changes : 15.26%   |   Max Drawdown : -45.23%   |   
↳ Env 0 : 032 :  2495993   |   368:26:42   |   Epsilon :  0.00%
saving
----------
{'batch_size': 64, 'train_every': 128}
----------
___________________________________________ VALIDATION ___________________________________________
Market Return : -64.43%   |   Portfolio Return : 136.86%   |   Position Changes : 14.81%   |   Max Drawdown : -53.51%   |   
↳ Env 0 : 032 :  2495993   |   368:29:32   |   Epsilon :  0.00%
saving
----------
{'batch_size': 64, 'train_every': 256}
----------
___________________________________________ VALIDATION ___________________________________________
Market Return : -64.43%   |   Portfolio Return : 196.78%   |   Position Changes : 16.34%   |   Max Drawdown : -36.20%   |   
↳ Env 0 : 032 :  2495993   |   368:31:46   |   Epsilon :  0.00%
saving
----------
{'batch_size': 128, 'train_every': 64}
----------
_______________________________

In [4]:
agent = reload_agent()
agent.epsilon_function = lambda episode, step : 0

env = get_env("2017")

evaluation(train=False)

list_of_files = glob.glob('render_logs/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
os.rename(latest_file, 'render_logs/benchmark_no_training.pkl')

del agent

___________________________________________ VALIDATION ___________________________________________
Market Return : -64.43%   |   Portfolio Return : 181.98%   |   Position Changes : 16.17%   |   Max Drawdown : -37.15%   |   
↳ Env 0 : 032 :  2480000   |   369:22:56   |   Epsilon :  0.00%
saving


In [2]:
for i in range(1):
    agent = reload_agent(
        batch_size=256,
        train_every=64
    )
    agent.epsilon_function = lambda episode, step : 0
    
    env = get_env("2017")
    
    evaluation()
    
    del agent

2024-06-05 11:05:10.028033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 11:05:10.090604: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 11:05:10.090726: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 11:05:10.092037: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 11:05:10.092108: I external/local_xla/xla/stream_executor

___________________________________________ VALIDATION ___________________________________________


2024-06-05 11:05:48.816743: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-06-05 11:05:56.243914: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3be3508190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 11:05:56.243956: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 980 Ti, Compute Capability 5.2
2024-06-05 11:05:56.248534: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1717578356.312088   31314 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Market Return : -64.43%   |   Portfolio Return : 198.68%   |   Position Changes : 13.54%   |   Max Drawdown : -37.61%   |   
↳ Env 0 : 032 :  2495993   |   380:54:52   |   Epsilon :  0.00%
saving
